In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning
from pytorch_lightning import loggers, Trainer

from dhdrnet.lightning_model import DHDRNet
from dhdrnet.util import DATA_DIR
%matplotlib inline
%load_ext autoreload
%autoreload 2

print(DATA_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/smsx/src/DHDRNet/data


In [12]:
model = DHDRNet()

trainer = Trainer()


INFO:lightning:GPU available: False, used: False


In [13]:
trainer.fit(model)



INFO:lightning:
   | Name                               | Type            | Params
-------------------------------------------------------------------
0  | inc                                | DoubleConv      | 38 K  
1  | inc.double_conv                    | Sequential      | 38 K  
2  | inc.double_conv.0                  | Conv2d          | 1 K   
3  | inc.double_conv.1                  | BatchNorm2d     | 128   
4  | inc.double_conv.2                  | ReLU            | 0     
5  | inc.double_conv.3                  | Conv2d          | 36 K  
6  | inc.double_conv.4                  | BatchNorm2d     | 128   
7  | inc.double_conv.5                  | ReLU            | 0     
8  | down1                              | Down            | 221 K 
9  | down1.maxpool_conv                 | Sequential      | 221 K 
10 | down1.maxpool_conv.0               | MaxPool2d       | 0     
11 | down1.maxpool_conv.1               | DoubleConv      | 221 K 
12 | down1.maxpool_conv.1.double_conv   | Seq

TypeError: only integer tensors of a single element can be converted to an index